In [1]:
import torch

In [2]:
x = torch.ones(5)  #输入张量
y = torch.zeros(3)   #定义标签
w = torch.randn(5,3,requires_grad = True)
b = torch.randn(3,requires_grad = True)    # w b是两个参数，requires_grad = True需要计算梯度
z = torch.matmul(x,w)+b   #预测输出
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

训练神经网络的过程，就是不断地去改变这个网络参数的值比如w b，使得网络的预测输出z越接近真实目标y越好。一般就是用梯度后向传播算法，根据z和y的距离得出loss，再根据loss得出loss关于w和loss关于b的梯度，再根据梯度下降算法更新w和b（10 15分钟有问题

In [3]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss = ',loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x000001DF74BAA730>
Gradient function for loss =  <BinaryCrossEntropyWithLogitsBackward0 object at 0x000001DF67BDB460>


In [4]:
loss.backward()  #backward()是属于tensor类的一个函数。前提loss是标量。对于可以计算梯度的父节点都会计算出梯度
print(w.grad)
print(b.grad) #打印所有计算出梯度父节点的梯度，只能获取requires_grad = True参数的梯度

tensor([[0.3321, 0.3091, 0.3052],
        [0.3321, 0.3091, 0.3052],
        [0.3321, 0.3091, 0.3052],
        [0.3321, 0.3091, 0.3052],
        [0.3321, 0.3091, 0.3052]])
tensor([0.3321, 0.3091, 0.3052])


In [6]:
z = torch.matmul(x,w)+b  #只要父节点中有一个requires_grad为true，那么当前这个节点为true
print(z.requires_grad)

with torch.no_grad():    #torch.no_grad()是上下文管理器，使处在上下文的操作不需要计算梯度，使requires_grad为false
    z = torch.matmul(x,w)+b
print(z.requires_grad)

True
False


In [7]:
z = torch.matmul(x,w)+b
z_det = z.detach()  ##分离，调用该节点的detach函数，得到的新节点也会false
print(z_det.requires_grad)

False


In [8]:
inp = torch.eye(5,requires_grad = True)  
out = (inp+1).pow(2)                     #先加法操作，再平方操作
out.backward(torch.ones_like(inp),retain_graph=True)  #传入一个全1的矩阵充当vector v
print("First call\n",inp.grad)
out.backward(torch.ones_like(inp),retain_graph=True) #retain_graph=True目的是进行第二次调用
print("\nSecond call\n",inp.grad) 
inp.grad.zero_()
out.backward(torch.ones_like(inp),retain_graph=True)
print("\nCall after zeroing gradients\n",inp.grad)
##在每一步更新完参数后，我们需要对参数的梯度置零，不需要手动操作，调用优化器的置零操作即可

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
